# My notebook

In [1]:
import pandas as pd
import json
from tqdm.notebook import tqdm
tqdm.pandas()

In [18]:
with open("msample.json", "r") as f:
  items = [json.loads(line) for line in f]
  df = pd.DataFrame({
      "id": [x["_source"]["id"] for x in items],
      "name": [x["_source"]["name"] for x in items],
      "entype": [x["_source"]["entype"] for x in items],
  })

#print(df.head())

In [17]:
#df["entype"].value_counts()

In [4]:
!pip install -U sentence-transformers

You should consider upgrading via the '/Users/nicolo/.pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [6]:
df['Embeddings'] = df['name'].progress_apply(lambda x: model.encode(x))

  0%|          | 0/13545 [00:00<?, ?it/s]

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
def get_top5_embedding_rows(user_input):

    # convert user_input to embeddings

    vect2 = model.encode(user_input).reshape(1,-1)

    # make a copy of original dataframe
    df_temp = df.copy()




    df_temp['similarity'] = df_temp['Embeddings'].progress_apply(lambda vect1:cosine_similarity(vect1.reshape(1,-1), vect2)[0][0])


    top_5 = df_temp.sort_values('similarity',ascending=False).head(10)



    return top_5

In [9]:
get_top5_embedding_rows('I want to watch free movies that are funny')

  0%|          | 0/13545 [00:00<?, ?it/s]

,id,name,entype,Embeddings,similarity
8657,7542516020512670170,Free Comedy Movies,menu,"[0.17548315, 0.2623958, 0.81223214, 0.15351374...",0.810469
5343,6439756319852725170,Free Comedy Movies,menu,"[0.17548315, 0.2623958, 0.81223214, 0.15351374...",0.810469
10882,8315844297383158170,Free Scary Movies,menu,"[-0.12686095, 0.20544545, 0.6543064, 0.7497650...",0.808080
2423,5431224808014449170,Free Scary Movies,menu,"[-0.12686095, 0.20544545, 0.6543064, 0.7497650...",0.808080
3552,5812120936160627170,Free Scary Movies,menu,"[-0.12686095, 0.20544545, 0.6543064, 0.7497650...",0.808080
8353,7447048591127655170,Free kids movies,menu,"[0.5515677, -0.47705546, 0.9623705, 0.8922556,...",0.785216
3601,5831846450510580170,Free Kids Movies,menu,"[0.5515677, -0.47705546, 0.9623705, 0.8922556,...",0.785216
3595,5830120021381495170,Free kids movies,menu,"[0.5515677, -0.47705546, 0.9623705, 0.8922556,...",0.785216
10224,8080712748113503170,Free Kids Movies,menu,"[0.5515677, -0.47705546, 0.9623705, 0.8922556,...",0.785216
12375,8812146009735420170,Free Kids Movies,menu,"[0.5515677, -0.47705546, 0.9623705, 0.8922556,...",0.785216


# Desireble output with keras model

In [ ]:

        
class MEModel(Model):
    def __init__(self):
        super().__init__()
        
        # Define layers
        entity_input = Input(shape=(1,), name='entity_string', dtype=tf.string)
        candidate_input = Input(shape=(None,), name='candidate_string', dtype=tf.string)
        embedding_layer = UniversalEmbeddingLayer()
        dot_layer = DotProductLayer()
        
        # Construct model
        entity_embedding = embedding_layer(entity_input)
        candidate_embeddings = embedding_layer(candidate_input)
        score = tf.squeeze(dot_layer([entity_embedding, candidate_embeddings]))
    
        self.model = Model(inputs=[entity_input, candidate_input], outputs=score)
        
        # Define TensorFlow functions
        @tf.function(input_signature=[[tf.TensorSpec(shape=(None, 1), dtype=tf.string),
                                       tf.TensorSpec(shape=(None, None), dtype=tf.string)]])
        def resolve(input_tensors: List[tf.Tensor]) -> List[tf.Tensor]:
            return self.model(input_tensors)
        
        self.resolve = resolve
    
    def call(self, inputs, training=None, mask=None):
        return self.resolve(inputs)
    
me_model = MEModel()

In [ ]:
tf.saved_model.save(me_model, SAVE_PATH, signature={"resolve": me_model.resolve})